# HuggingFace로 기계번역 구현하기

이번에는 HuggingFace로 기계번역 모델을 학습해 볼 것입니다.
먼저 필요한 library들을 설치하고 import합시다.


In [1]:
import random
import evaluate
import numpy as np

from datasets import load_dataset
from transformers import AutoTokenizer

## Dataset 준비

기계번역을 위한 dataset을 준비하겠습니다.
Dataset은 `Helsinki-NLP/opus-100`을 활용합니다.


In [29]:
books = load_dataset("Helsinki-NLP/opus-100", "en-ko")
books

DatasetDict({
    test: Dataset({
        features: ['translation'],
        num_rows: 2000
    })
    train: Dataset({
        features: ['translation'],
        num_rows: 1000000
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 2000
    })
})

In [30]:
ds = load_dataset("obov/hf-aihub-sample-translate-en-ko")

README.md:   0%|          | 0.00/595 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.89M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/213k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/229k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/19098 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1009 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1062 [00:00<?, ? examples/s]

In [31]:
ds

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 19098
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 1009
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 1062
    })
})

보시다시피 각 data는 영어 문장을 `en`에, 한국어 문장을 `ko`에 저장하고 있습니다.

이번에는 tokenizer를 불러와 data를 미리 tokenize 하겠습니다.


In [3]:
source_lang = "en"
target_lang = "ko"
prefix = "translate English to Korean: "
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-tc-big-en-ko")


def preprocess_function(data):
    inputs = [prefix + text[source_lang] for text in data["translation"]]
    targets = [text[target_lang] for text in data["translation"]]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=128, truncation=True
    )
    return model_inputs

tokenizer_config.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/790k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/815k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/959k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

기계번역과 같이 입력과 출력을 모두 tokenize하는 경우에는 출력 text를 `tokenizer`의 `text_target`인자로 넘겨주면 됩니다. 이처럼 기계 번역 dataset을 불러오는 것은 imdb dataset과 별반 다르지 않습니다.

마지막으로 `data_collator`를 다음과 같이 구현합니다.


In [32]:
tokenized = ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/19098 [00:00<?, ? examples/s]

Map:   0%|          | 0/1009 [00:00<?, ? examples/s]

Map:   0%|          | 0/1062 [00:00<?, ? examples/s]

In [44]:
tokenized["train"][:10]["translation"]

[{'en': 'Professor Nonaka, who created the knowledge creation theory, explains the knowledge creation process as a \'spiral process\' through the "SECI model".',
  'ko': '지식 창조 이론을 창안한 노나카 교수는 "SECI모델"을 통해 지식 창조 과정을 `나선형 프로세스라고 설명하고 있다.'},
 {'en': 'First, among the educational cases aiming for integration or convergence between various genre arts, cases using metaverse platforms, hardware, and software were selected.',
  'ko': '첫 번째는 다양한 장르 예술 간 통합 또는 융합 예술을 지향하는 교육 사례 중 메타버스 플랫폼, 하드웨어, 소프트웨어 등을 활용한 사례를 선택하였다.'},
 {'en': 'Students recognize play as a teaching and learning framework for expression activities.',
  'ko': '학생들이 놀이를 표현 활동의 교수·학습 틀로 인식하고 있다.'},
 {'en': "The Korean people's attitude to detect the risk of fire, that is, awareness of fire, is relatively high at 74.4%, but only 27.8% have fire extinguishers.",
  'ko': '우리 국민은 화재의 위험도를 감지하는 태도, 즉 화재에 대한 인식은 74.4%로 비교적 높은 편이나 소화기 비치 정도는 27.8%에 지나지 않는다.'},
 {'en': 'An image map related to emotional design was created to extract emo

In [4]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="google-t5/t5-small")

## Model 구현

이번에는 기계번역을 위한 sequence-to-sequence 모델을 구현하겠습니다.


In [5]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained("google-t5/t5-small")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Sequence-to-sequence 모델은 위와 같이 `AutoModelForSeq2SeqLM`을 활용하면 됩니다.
Pre-trained 모델로 `google-t5/t5-small`이라는 것을 사용하고 있습니다.
이 코드 하나로 우리는 기계 번역과 같은 문제를 풀 수 있는 encoder-decoder 구조의 모델을 구현할 수 있습니다.


## 학습

마지막으로 학습 코드를 구현하겠습니다.
학습은 감정 분석과 똑같이 `training_args`를 정의하고 `Trainer`로 이전에 구현한 것들을 모두 수합하면 됩니다.


In [34]:
from callback_plot import PlotCallback
from callback_best_model import BestModelCallback

training_args = Seq2SeqTrainingArguments(
    output_dir="hf_mt",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    weight_decay=0.01,  # 첫 주차 때 배운 weight decay를 조절하는 hyper-parameter입니다.
    save_total_limit=1,
    num_train_epochs=1,
    predict_with_generate=True,  # 실제로 평가를 진행할 때는 내부적으로 정의된 beam search 등을 활용하여 text를 생성합니다.
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[
        PlotCallback(),
        BestModelCallback(),
    ],
)

마지막으로 학습하면 다음과 같습니다.


In [36]:
trainer.train()
trainer.save_model()

  0%|          | 0/1194 [00:00<?, ?it/s]

{'loss': 8.3366, 'grad_norm': 1.1837365627288818, 'learning_rate': 1.1624790619765495e-05, 'epoch': 0.42}
{'loss': 8.1945, 'grad_norm': 1.005014419555664, 'learning_rate': 3.2495812395309884e-06, 'epoch': 0.84}
{'train_runtime': 426.316, 'train_samples_per_second': 44.798, 'train_steps_per_second': 2.801, 'train_loss': 8.250422645453831, 'epoch': 1.0}


학습 결과를 보면 다음과 같습니다.


In [41]:
from transformers import pipeline

text = "emotional design was created"
translator = pipeline(
    "translation",
    model="./hf_mt",
    max_new_tokens=128,
    device="mps",
    tokenizer=tokenizer,
)
translator(text)

[{'translation_text': '.을을 German German:.의.을을 German:.의.의.을을 German:.의.의.의.의.의.을을 German:.의.의.의.의.의.의.의.의 German:.의.의.의.의.의.의.의 German:.의.의.의.의.의.의.의.의.의.의 German:.의.의.의.의.의.의.의.의.의.의.의.의의의의의의.의.의.의'}]

보시다시피 성능이 그렇게 좋지 않습니다. 아마 더 많은 학습 시간을 요구하는 듯 합니다.

하지만 중요한 것은 data부터 모델, 학습 코드 구현까지 매우 쉽게 했다는 것입니다.
